### *Импорты* ###

In [1]:
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model

from datetime import datetime, timedelta

### *Загрузка файла и стандартизирование данных* ###

In [2]:
data = pd.read_csv('../ann/../../db/basel(1940-2023).csv')

scaler_temperature = joblib.load('scalers/scaler_temperature.joblib')

### *Загрузка моделей* ###

In [3]:
model_temperature = load_model('best_models/best_model_temperature.keras')

In [4]:
def predict_weather_next_hours(date):
    year, month, day, hour = map(int, date.split('.'))                                     

    current_hour_data = get_current_hour_data(year, month, day, hour)
    last_5_hours_data = get_last_5_hours_data(year, month, day, hour)
    last_5_days_data = get_last_5_days_data(year, month, day, hour)

    temperature_prediction = predict_temperature(current_hour_data, last_5_hours_data, last_5_days_data)
    print('Predicted value: {}'.format(temperature_prediction))

In [5]:
def get_current_hour_data(year, month, day, hour):
    current_hour_data = data[(data['Year'] == year) &
                             (data['Month'] == month) &
                             (data['Day'] == day) &
                             (data['Hour'] == hour)]
    
    # Если данные о текущем часе неизвестны, будут взяты данные прошлого года
    while current_hour_data.empty:
        year -= 1
        
        current_hour_data = data[(data['Year'] == year) &
                         (data['Month'] == month) &
                         (data['Day'] == day) &
                         (data['Hour'] == hour)]
        
    return current_hour_data

In [6]:
def get_last_5_hours_data(year, month, day, hour):
    target_date = datetime(year, month, day, hour)
    target_dates = [target_date - timedelta(hours=i) for i in range(1, 6)]

    last_5_hours_data = pd.concat([data[(data['Year'] == date.year) &
                                       (data['Month'] == date.month) &
                                       (data['Day'] == date.day) &
                                       (data['Hour'] == date.hour)] for date in target_dates], ignore_index=True)
    
    # Если данные о последних 5 часах перед предсказываемой датой неизвестны, будут взяты данные прошлого года
    while last_5_hours_data.empty:
        target_date -= timedelta(days=365)
        target_dates = [target_date - timedelta(hours=i) for i in range(1, 6)]

        last_5_hours_data = pd.concat([data[(data['Year'] == date.year) &
                                           (data['Month'] == date.month) &
                                           (data['Day'] == date.day) &
                                           (data['Hour'] == date.hour)] for date in target_dates], ignore_index=True)

    return last_5_hours_data

In [7]:
def get_last_5_days_data(year, month, day, hour):
    target_date = datetime(year, month, day, hour)
    target_dates = [target_date - timedelta(days=i) for i in range(1, 6)]
    
    last_5_days_data = pd.concat([data[(data['Year'] == date.year) &
                                       (data['Month'] == date.month) &
                                       (data['Day'] == date.day) &
                                       (data['Hour'] == date.hour)] for date in target_dates], ignore_index=True)
    
    # Если данные о последних 5 днях перед предсказываемой датой неизвестны, будут взяты данные прошлого года
    while last_5_days_data.empty:
        target_date -= timedelta(days=365)
        target_dates = [target_date - timedelta(days=i) for i in range(1, 6)]

        last_5_days_data = pd.concat([data[(data['Year'] == date.year) &
                                           (data['Month'] == date.month) &
                                           (data['Day'] == date.day) &
                                           (data['Hour'] == date.hour)] for date in target_dates], ignore_index=True)

    return last_5_days_data

In [8]:
def predict_temperature(current_hour_data, last_5_hours_data, last_5_days_data):
    current_hour_data = current_hour_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    last_5_hours_data = last_5_hours_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    last_5_days_data = last_5_days_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    
    middle_point = (current_hour_data + last_5_hours_data.mean() + last_5_days_data.mean()) / 3
    middle_point_scaled = scaler_temperature.transform(middle_point)
    
    prediction = model_temperature.predict(middle_point_scaled)
    return prediction

In [9]:
for i in range(24):
    predict_weather_next_hours('2023.12.3.{}'.format(i))

1/1 [==============================] - 0s 72ms/step
Predicted value: [[3.7818642]]
1/1 [==============================] - 0s 0s/step
Predicted value: [[3.9053988]]
1/1 [==============================] - 0s 5ms/step
Predicted value: [[3.2977273]]
1/1 [==============================] - 0s 16ms/step
Predicted value: [[2.8577976]]
1/1 [==============================] - 0s 24ms/step
Predicted value: [[2.4400883]]
1/1 [==============================] - 0s 6ms/step
Predicted value: [[2.7357137]]
1/1 [==============================] - 0s 16ms/step
Predicted value: [[2.59143]]
1/1 [==============================] - 0s 16ms/step
Predicted value: [[2.2057095]]
1/1 [==============================] - 0s 16ms/step
Predicted value: [[2.1554718]]
1/1 [==============================] - 0s 0s/step
Predicted value: [[2.036762]]
1/1 [==============================] - 0s 16ms/step
Predicted value: [[2.159098]]
1/1 [==============================] - 0s 16ms/step
Predicted value: [[1.9217606]]
1/1 [=========